In [ ]:
#Importing Necessary Libraries
import ee
import geopandas as gpd

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-subodhpaudel123')

In [ ]:
#Defining the Export Function
def export_dynamic_world(year,region,folder='Dynamicworldexports'):
  start_date= ee.Date.fromYMD(year,1,1)
  end_date= start_date.advance(1,'year')

  dw= ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
  .filterDate(start_date,end_date)\
  .filterBounds(region)\
  .median().clip(region)

  dw_image = dw.select('label').toInt()

  task= ee.batch.Export.image.toDrive(
      image= dw_image,
      description= f"Hetauda{year}",
      scale= 10,
      region=region,
      maxPixels=1e13,
      crs='EPSG:4326'
  )
  task.start()
  print(f"Exporting Dynamic World data for Hetauda {year} to Google Drive...")
#Defining the Region of Interest (ROI)
gdf= gpd.read_file('/content/Hetauda.shp')
geo_json = gdf.__geo_interface__
region= ee.Geometry.Polygon(geo_json['features'][0]['geometry']['coordinates'])
#Running the Export for Specific Years
years= range(2016,2025)
for year in years:
  export_dynamic_world(year,region)